In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lower
from pyspark.sql.functions import col
from pyspark.sql.functions import (col, when)
from pyspark.sql.functions import month, max, date_format
import calendar

In [2]:
spark = SparkSession.builder.appName("test_app").getOrCreate()

In [3]:
df = spark.read.csv("complete.csv", header=True, inferSchema=True)

In [4]:
df = df.withColumn("State", lower(df["Name of State / UT"]))

In [5]:
df.show()

+----------+------------------+--------+---------+---------------------+-----+-------------------------+---------+----------+-------------+------+
|      Date|Name of State / UT|Latitude|Longitude|Total Confirmed cases|Death|Cured/Discharged/Migrated|New cases|New deaths|New recovered| State|
+----------+------------------+--------+---------+---------------------+-----+-------------------------+---------+----------+-------------+------+
|2020-01-30|            Kerala| 10.8505|  76.2711|                  1.0|    0|                      0.0|        0|         0|            0|kerala|
|2020-01-31|            Kerala| 10.8505|  76.2711|                  1.0|    0|                      0.0|        0|         0|            0|kerala|
|2020-02-01|            Kerala| 10.8505|  76.2711|                  2.0|    0|                      0.0|        1|         0|            0|kerala|
|2020-02-02|            Kerala| 10.8505|  76.2711|                  3.0|    0|                      0.0|        1|    

Day with the greater number of COVID Cases

In [6]:
max_cases_day = df.orderBy(col("Total Confirmed cases").desc()).first()
print(max_cases_day['Date'])

2020-08-06


State with the Second-Largest Number of COVID Cases

In [7]:
second_max_cases_state = df.orderBy(col("Total Confirmed cases").desc()).distinct().collect()[1]
print(second_max_cases_state['State'])

punjab


Union Territory with the Least Number of Deaths

In [8]:
ut_df = df.filter(col("Name of State / UT").contains("Union Territory"))
ut_least_deaths = ut_df.orderBy(col("Death").asc()).first()
print(ut_least_deaths['State'])

union territory of ladakh


State with the Lowest Death to Total Confirmed Cases Ratio

In [9]:
df = df.withColumn("DeathToConfirmedRatio", col("Death") / col("Total Confirmed cases"))
min_ratio_state = df.orderBy(col("DeathToConfirmedRatio").asc()).first()
print(min_ratio_state['State'])

puducherry


Month with the Most Newer Recovered Cases

In [10]:
df = df.withColumn("Month", month(col("Date")))
monthly_recovered = df.groupBy("Month").sum("New recovered")
max_recovered_month = monthly_recovered.orderBy(col("sum(New recovered)").desc()).first()
month_name = calendar.month_name[max_recovered_month['Month']]
print(month_name)

July
